In [1]:
import h5py
import tensorflow as tf

2024-12-09 11:22:13.906988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-09 11:22:13.907032: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
import tensorflow_probability as tfp
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    print(tf.shape(counts_per_example))
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    log_probs = tf.reduce_sum(dist.log_prob(true_counts))
    print(tf.shape(log_probs))
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.cast(tf.shape(true_counts)[0], dtype=tf.float32))




In [4]:
import torch.nn as nn
import torch.nn.functional as F
class MultinomialNLLLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits, true_counts):
        # Compute log probabilities for each category
        true_counts = torch.clamp(true_counts, min=1e-8)  # Avoid log(0)
        log_probs = F.log_softmax(logits, dim=-1)
        log_likelihood = torch.sum(true_counts * log_probs, dim=-1)
        total_counts = true_counts.sum(dim=-1)
        log_gamma_term = torch.lgamma(total_counts + 1)
        log_gamma_counts = torch.lgamma(true_counts + 1).sum(dim=-1)
        log_likelihood = log_likelihood + log_gamma_term - log_gamma_counts
        return -torch.mean(log_likelihood)

In [5]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('../..')))

from models._data import ChromatinDataset
from models._model import CBPLTrainer

In [6]:
train_chrs = ["chr2",
    "chr4",
    "chr5",
    "chr7",
    "chr9",
    "chr10",
    "chr11",
    "chr12",
    "chr13",
    "chr14",
    "chr15",
    "chr16",
    "chr17",
    "chr18",
    "chr19",
    "chr21",
    "chr22",
    "chrX",
    "chrY"]

In [7]:
config = {
    "peak_regions": "/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/pd_data/Microglia_peak_set_2.bed",
    "nonpeak_regions": "/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.negatives.adjusted.bed",
    "genome_fasta": "/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/data/downloads/hg38.fa",
    "cts_bw_file": "/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/pd_data/nd_Microglia_merge.bw",
    "negative_sampling_ratio": 0,
    "train_size": 0.6,
    "batch_size": 32,
    "filters": 512,
    "n_dil_layers": 8,
    "conv1_kernel_size": 21,
    "profile_kernel_size": 71,
    "dilation_kernel_size": 3,
    "input_seq_len": 2114,
    "out_pred_len": 1000,
    "dropout_rate": 0.0,
    "learning_rate": 0.001,
    "train_chrs": train_chrs,
    "valid_chrs": ["chr1"],
    "seq_focus_len": 500,
    "use_cpu": False,
    "alpha" : 1,
    "checkpoint_path": None,
    "flavor" : None,
    "project": "cbpl_new_microglia"
}



In [8]:
trainer =  CBPLTrainer(config)

Read in bed file of 151351 regions
Read in bed file of 16900 regions
Loaded 151351 peak regions and 0 non-peak regions
Calculating average count for combined loss weight
Average count per training peak = 135457.5
cpu


/wynton/home/corces/vishvak/miniforge3/envs/chrombpnet/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [9]:
for batch in trainer.train_dataloader:
    inputs, targets = batch
    break  

In [10]:
pytorch_nll = MultinomialNLLLoss()

In [11]:
outputs = trainer.model.forward_test(inputs)

starting size:torch.Size([32, 4, 2114])
after first convolution:torch.Size([32, 512, 2094])
after 1th dilation:torch.Size([32, 512, 2090])
after 1th crop:torch.Size([32, 512, 2090])
after 2th dilation:torch.Size([32, 512, 2082])
after 2th crop:torch.Size([32, 512, 2082])
after 3th dilation:torch.Size([32, 512, 2066])
after 3th crop:torch.Size([32, 512, 2066])
after 4th dilation:torch.Size([32, 512, 2034])
after 4th crop:torch.Size([32, 512, 2034])
after 5th dilation:torch.Size([32, 512, 1970])
after 5th crop:torch.Size([32, 512, 1970])
after 6th dilation:torch.Size([32, 512, 1842])
after 6th crop:torch.Size([32, 512, 1842])
after 7th dilation:torch.Size([32, 512, 1586])
after 7th crop:torch.Size([32, 512, 1586])
after 8th dilation:torch.Size([32, 512, 1074])
after 8th crop:torch.Size([32, 512, 1074])
Profile prediction shape: torch.Size([32, 1000])
Count prediction shape: torch.Size([32, 1])


In [12]:
import tensorflow as tf
import tensorflow_probability as tfp


#from https://github.com/kundajelab/basepair/blob/cda0875571066343cdf90aed031f7c51714d991a/basepair/losses.py#L87
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.cast(tf.shape(true_counts)[0], dtype=tf.float32))




In [13]:
tf_targets = tf.convert_to_tensor(targets[1].detach().numpy())
tf_outputs = tf.convert_to_tensor(outputs[1].detach().numpy())

2024-12-03 13:35:08.495966: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-03 13:35:08.496877: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-12-03 13:35:08.497680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-12-03 13:35:08.498528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2024-12-03 13:35:08.500478: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [14]:
tf_input = tf.convert_to_tensor(inputs.numpy())

In [16]:
multinomial_nll(tf_targets,tf_outputs)

<tf.Tensor: shape=(), dtype=float32, numpy=44542.938>

In [17]:
dist = tfp.distributions.Multinomial(total_count=tf.reduce_sum(tf_targets, axis=-1),
                                         logits=tf_outputs)

In [125]:
dist.log_prob(tf_targets)

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([-206860.25  ,  -70000.375 ,   -9350.875 ,  -31095.25  ,
       -105474.125 ,  -54320.875 ,  -84637.5   ,  -10018.797 ,
        -18328.75  ,  -15417.531 ,  -45188.438 ,   -6810.375 ,
        -20707.688 ,  -28331.375 , -191278.    ,  -35936.188 ,
        -44314.    ,   -5278.3438, -262125.    ,  -74644.125 ,
        -16878.125 ,  -22152.375 , -119300.    ,   -5027.5156,
        -22823.438 ,   -9276.4375,  -22894.188 ,  -51361.    ,
         -7402.9375,  -66321.31  ,  -44446.75  ,  -21035.5   ],
      dtype=float32)>

In [27]:
from torchmetrics.regression import PearsonCorrCoef, SpearmanCorrCoef

In [19]:
pearsonr =PearsonCorrCoef(num_outputs=32)

In [28]:
spearmanr = SpearmanCorrCoef(num_outputs=32)

/wynton/home/corces/vishvak/miniforge3/envs/chrombpnet/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [141]:
pearsonr(torch.from_numpy(tf_targets.numpy()).T,torch.from_numpy(tf_outputs.numpy()).T)

tensor([ 0.0402,  0.0260,  0.0657,  0.0198, -0.0544, -0.0413,  0.0310,  0.0358,
        -0.0217,  0.0934,  0.0363, -0.0154,  0.0559,  0.1400, -0.1085,  0.0459,
        -0.0111, -0.0063, -0.0259,  0.0175,  0.0123,  0.0198, -0.0224, -0.0130,
         0.0072,  0.0175, -0.0419, -0.0231,  0.0165, -0.1030, -0.0140, -0.0536])

In [117]:
tf_outputs

<tf.Tensor: shape=(32, 1000), dtype=float32, numpy=
array([[0.58572733, 0.8405155 , 0.8844119 , ..., 0.612815  , 0.6666953 ,
        0.72798973],
       [0.32442287, 0.65844226, 0.74438804, ..., 0.7804712 , 0.80494523,
        0.7567023 ],
       [0.6387386 , 0.6611079 , 0.6144929 , ..., 0.5243706 , 0.9197526 ,
        0.6190738 ],
       ...,
       [0.7179155 , 0.83784884, 0.6963693 , ..., 0.77240825, 0.77436274,
        0.8944525 ],
       [0.7897969 , 0.7258214 , 0.7729998 , ..., 0.5469245 , 0.83745   ,
        0.7259275 ],
       [0.8847068 , 0.7565618 , 0.3998531 , ..., 0.5254934 , 0.8618141 ,
        0.74292195]], dtype=float32)>

In [113]:
pytorch_nll(outputs[1],targets[1])

tensor(54032.3711, grad_fn=<NegBackward0>)

In [21]:
tf_model = tf.keras.models.load_model('/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/pd_data/microglia_model.h5')


In [22]:
output = tf_model.predict(tf_input)

In [23]:
multinomial_nll(tf_targets,output[0])

<tf.Tensor: shape=(), dtype=float32, numpy=26408.703>

In [26]:
pearsonr(torch.from_numpy(tf_targets.numpy()).T,torch.from_numpy(output[0]).T)

tensor([ 0.7434,  0.7201,  0.1712,  0.7233,  0.7153,  0.8335,  0.1701,  0.8567,
         0.8875,  0.2959,  0.8642, -0.0159,  0.7315,  0.6346,  0.2625,  0.7861,
         0.3647,  0.6048,  0.6864,  0.5203,  0.6476,  0.8209,  0.7014,  0.7393,
         0.8539,  0.3961,  0.8314,  0.5989,  0.7010,  0.6842,  0.7242,  0.8672])

In [29]:
spearmanr(torch.from_numpy(tf_targets.numpy()).T,torch.from_numpy(output[0]).T)

tensor([0.7781, 0.6257, 0.1803, 0.6964, 0.7338, 0.7448, 0.1725, 0.8470, 0.8967,
        0.2291, 0.8479, 0.0691, 0.6774, 0.6110, 0.1892, 0.7861, 0.4045, 0.6663,
        0.8028, 0.5376, 0.4868, 0.8195, 0.7334, 0.7770, 0.7628, 0.3412, 0.8699,
        0.6559, 0.6039, 0.7476, 0.5493, 0.8364])

In [116]:
tf_targets

<tf.Tensor: shape=(32, 1000), dtype=float32, numpy=
array([[ 45.,  46.,  48., ...,  75.,  75.,  76.],
       [ 63.,  63.,  62., ...,   7.,   7.,   7.],
       [ 17.,  17.,  17., ...,  12.,  12.,  12.],
       ...,
       [ 26.,  26.,  27., ...,  61.,  61.,  65.],
       [ 27.,  23.,  23., ...,  96.,  97.,  97.],
       [171., 182., 182., ...,  91.,  91.,  93.]], dtype=float32)>

In [90]:
output[1]

array([[6.9208736],
       [5.997568 ],
       [7.559485 ],
       [6.5625315],
       [6.673019 ],
       [7.1238184],
       [6.8574533],
       [6.983832 ],
       [8.067585 ],
       [7.1695156],
       [7.6921005],
       [6.895475 ],
       [8.836632 ],
       [6.688803 ],
       [9.027556 ],
       [7.5389924],
       [8.572815 ],
       [7.6638603],
       [7.540666 ],
       [6.5197425],
       [6.483042 ],
       [6.1656632],
       [7.498359 ],
       [8.099946 ],
       [6.5437503],
       [6.887114 ],
       [7.012876 ],
       [7.385822 ],
       [6.803893 ],
       [6.666781 ],
       [6.840408 ],
       [7.825834 ]], dtype=float32)

In [59]:
tf.shape(tf_input)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([  32, 2214,    4], dtype=int32)>